In [1]:
## The neuropath data at Emory ADRC is currently not particularly well organized
## and naming schema is inconsistent.  Working on developing a schema 
import girder_client
import os,sys, json
import dagSecrets as dgs
from collections import Counter
import pandas as pd
import pickle
import brain_region_maps as brm
%load_ext autoreload
%autoreload 2
import adrcSchemaHelpers as hlprs
gc = girder_client.GirderClient(apiUrl=dgs.apiUrl)
gc.authenticate(apiKey=dgs.apiToken)

{'_id': '5f3d80d2615f2a24eb8fee20'}

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
with open("npSampleJsonData.json","w") as fp:
    json.dump(npSlideDict,fp)



In [3]:
npPickleFile = 'npDemoData.pickle'
savePickle = True

if os.path.isfile(npPickleFile):
    with open(npPickleFile,"rb") as fp:
        npSlideDict=  pickle.load(fp)
else:
    npSlideDict = hlprs.getADRCcollection(gc)


        
    if savePickle:

        with open(npPickleFile,"wb") as fp:
            pickle.dump(npSlideDict,fp)

    

In [4]:
## meta.region  meta.stain  meta.subject
basicSlideInfo = []
mdKeys = ['region','stain','subject','brain_region']

for subjyr in npSlideDict:
    for s in npSlideDict[subjyr]:
        slideData = {'folderId': subjyr,'slideName':s['name'],'itemId':s['_id']}
        
        for k in mdKeys:
            if k in s['meta']:
                slideData[k] = s['meta'][k]
        basicSlideInfo.append(slideData)
pd.DataFrame(basicSlideInfo).to_csv("adrcInitialMetadataExport.csv")

In [5]:
npByYearCase = {}
for k in npSlideDict:
    try:
        (year,caseId) = k.split("_")
        if year not in npByYearCase:
            npByYearCase[year]= []
            
        npByYearCase[year].append(caseId)
        
    except:
        print("Weird pattern for",k)

Weird pattern for 2009_Gwinnett_consult_A09-09_micro_photos
Weird pattern for 2009_OS10_21_09
Weird pattern for 2017_OS_170124
Weird pattern for 2017_OS_170518


In [6]:
## Will shim to the preferred alias
stainAliasDict = { 'PTDP':'pTDP', 'HE':'HE','AB':'aBeta','AB':'AB','ptdp':'pTDP','ubiq':'Ubiq','H&E':'HE' }

## For control slides I make the region Control  and also set the controlSlide Flag... not sure what to do about the
## slideName..
def validateSlide( slideInfo, patientId):
    ## This will be a cleanup function that either validates data is meta.npSc,hema is available
    ## or tries to populate it from the existing and likely dirty metadata from either parsing the slide name
    ## or if meta.region or meta.stain are already populated..
    meta = slideInfo['meta']
    
    ## Will clean up the region and remove the a's
    ## Remove the A's if it happens to have been encoded in the region but only if it's not a 1A.. HMM
    
    if 'npSchema' in meta:
        pass
    else:
        ### First try and validate the region
        if 'region' in meta:
            blockID = meta['region']
            if blockID.startswith('A'):
                blockID = blockID.replace("A","")
            print(blockID)
            return ({'region':meta['region'],'blockID':blockID, 'subject':meta['subject'],'stain':meta['stain'], 'filename':slideInfo['name']})
        else:
            return ({'filename':slideInfo['name']})
            
#             print(meta['region'],meta['subject'])
            


In [62]:
from ipywidgets import Dropdown, interactive, HBox, interactive_output
from collections import Counter
npYearList = list(npByYearCase.keys())

dpYear = Dropdown(options=npYearList,description='Year')
dpCaseId = Dropdown(options=npByYearCase[npYearList[0]],description='CaseId')


def on_change(*args):
    value = dpYear.value
    dpCaseId.options = npByYearCase[value]
dpYear.observe(on_change, names='value')

def analyzeNpCase(year, caseId):
    ### This will perform an analysis of a folder of images to see the number of slides 
    ## And check the metadata
    npKey = "%s_%s" % ( year,caseId)
    slideList = npSlideDict[npKey]
    
    ## Keys of interest
    koi = ['region','regionId','subject','stain','Stain','Region']
    slideKeys = {}
    for k in koi:
        slideKeys[k]= []
    
    caseMeta = [] ## 
    d=caseMeta ## Exposing this as a global
    
    for s in slideList:
        meta = s['meta']
        for k in meta:
            if k in koi:
                slideKeys[k].append(meta[k])
        caseMeta.append( validateSlide(s,caseId) )
                
                
    ## Now go through and make things into counters so it's easier to see
    for k in koi:
        slideKeys[k] = Counter(slideKeys[k])
                
    print("Unique RegionId Count: %d \t Distinct SubjID: \t%d"% (len(slideKeys['region']),len(slideKeys['subject'])))
    print("Total slides found: %d" % len(slideList))  ## Will add in schema maps as well
    df = pd.DataFrame(caseMeta)
#     print(df)
    if 'region' in df.columns:
        print(df.region.value_counts())
    else:
        print("region tag not found..")
        print([s['name'] for s in slideList])

    
ui = HBox([dpYear,dpCaseId])
w = interactive_output(analyzeNpCase, {'year':dpYear, 'caseId':dpCaseId})
display(ui,w)

## Key I am interested in is npSchema going forward
## Need to figure out what to do with control slides..  will add a key to the Schema that's optional called controlSlide


Output()

In [44]:
brm.MAP['A18-07']

{'1': 'Hippocampus',
 '2': 'Amygdala',
 '3': 'Frontal cortex',
 '4': 'Temporal cortex',
 '5': 'Parietal cortex',
 '6': 'Occipital cortex',
 '7': 'Left perirolandic cortex',
 '8': 'Cingulate cortex',
 '9': 'Anterior basal ganglia and insular cortex',
 '10': 'Posterior basal ganglia',
 '11': 'Hypothalamus',
 '12': 'Thalamus, with subthalamic nucleus',
 '13': 'Midbrain',
 '14': 'Pons',
 '15': 'Medulla',
 '16': 'Cerebellum and dentate nucleus',
 '17': 'Upper spinal cord',
 '18': 'Mid-level spinal cord',
 '19': 'Lower spinal cord',
 '20': 'Dorsal root ganglia'}

'meh'